In [ ]:
from oot.oot import Oot
import os, random, requests, time

CACHE_ENABLED = True
SEED = None
LOGDIR = 'logs'
GAMEDIR = 'roms'
game = Oot(gamedir=GAMEDIR, logdir=LOGDIR)

In [ ]:
# Running this cell should start to finish randomize the actors in the rom
actor_prefix = 'En_Wf' # Only randomize among actors starting with this prefix
start_time = time.time()
seed = SEED if SEED else str(start_time)
setSeed(seed)
print("Seed: {}".format(seed) )

readData()
print("Data read, {} sec".format(time.time()-start_time))

rooms_list = getRoomNames()
rooms = generateRooms(rooms_list)
print("Room Info Generated, {} sec".format(time.time()-start_time))

# Generate a dict with actor/{object: associated_object, variable: var} pairs
actors_associated = {}
for key, room in rooms.items():
    for num, setup in room.setups.items():
        for actor in setup['actors']:
            if actor.object_name is not 'NA':
                actors_associated[actor.filename] = {
                    "object": actor.object_name,
                    "variable": actor.var
                }
available_actors = {k:a for k,a in actors_associated.items() if k.startswith(actor_prefix)}
print("Actors Associated, {} sec".format(time.time()-start_time))

spoilers = {}
for r, room in rooms.items():
    for n, setup in room.setups.items():
        key = '{}-{}'.format(r,n)
        room_spoiler = []
        spoilers[key] = room_spoiler
        for a, actor in enumerate(setup['actors']):
            if actor.filename in available_actors and actor.filename not in [rm["new"]["filename"] for rm in room_spoiler]:
                new_actor_name = random.choice(list(available_actors.keys()))
                old_actor_info = {"filename": actor.filename, "object": actor.object_name, "description": actor.description, "variable": actor.var}
                room.replaceActor(actor.filename, new_actor_name, new_var=available_actors[new_actor_name]['variable'])
                new_actor_info = {"filename": setup['actors'][a].filename, "object": setup['actors'][a].object_name, "description": setup['actors'][a].description, "variable": actor.var}
                room_spoiler.append({"old": old_actor_info, "new": new_actor_info})
        spoilers[r] = room_spoiler
print("Rooms Randomized, {} sec".format(time.time()-start_time))

# Print out spoilers
writeSpoilers(spoilers, "spoiler.log")
print("Replacement Log Written".format(time.time()-start_time))

writeData()
print("New ROM Written {}".format(time.time()-start_time))

In [ ]:
# Running this cell should start to finish randomize the actors within each setup
start_time = time.time()
seed = SEED if SEED else str(start_time)
setSeed(seed)
print("Seed: {}".format(seed) )

readData()
print("Data read, {} sec".format(time.time()-start_time))

rooms_list = getRoomNames()
rooms = generateRooms(rooms_list)
print("Room Info Generated, {} sec".format(time.time()-start_time))
        
def _writeSpoilers(sp, fn):
    with open(fn, "w") as f:
        for k,v in sp.items():
            f.write('{}\n'.format(k))
            for c in v:
                f.write('{} ({}) -> {} ({})\n'.format(c['old']['fn'], c['old']['var'], c['new']['fn'], c['new']['var']))
spoilers = {}
for r, room in rooms.items():
    for n, setup in room.setups.items():
        key = '{}-{}'.format(r,n)
        room_spoiler = []
        spoilers[key] = room_spoiler
        setup_actors = [ {"fn": a.filename, "var": a.var}
           for a in setup['actors']]
        new_setup_actors = list(setup_actors)
        random.shuffle(new_setup_actors)
        for a, actor in enumerate(setup['actors']):
            old = setup_actors[a]
            new = new_setup_actors[a]
            room.replaceActor(old['fn'], new['fn'], new_var=new['var'], absIndex=a, replaceObject=False, setup=n)
            room_spoiler.append({'old': old, 'new': new})
print("Rooms Randomized, {} sec".format(time.time()-start_time))

# Print out spoilers
_writeSpoilers(spoilers, "spoiler.log")
print("Replacement Log Written".format(time.time()-start_time))

writeData()
print("New ROM Written {}".format(time.time()-start_time))